# RunnablePassthrough

#### 필수 라이브러리 설치

In [79]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai

#### OpenAI API KEY 등록

In [80]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*******************************************"

In [102]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

#Runnable 생성
#해당 변수에 값을 지정할 경우, 병렬로 input값을 처리함
# passed = 패스 변수 설정
# extra = 패스 변수에 숫자 할당
# modified = 패스 변수 변경
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

#실행
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

## **Retrieval Example**

In [103]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#vector db 생성 
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)

#검색기 생성
retriever = vectorstore.as_retriever()

#프롬프트 생성
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

#모델 생성
model = ChatOpenAI()

#체인 생성
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

#체인 작동
retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'